![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/ONNX_Configs_in_Spark_NLP_AlbertForQuestionAnswering.ipynb)

## Setting ONNX configs in SparkNLP

Starting from Spark NLP 5.1.2, you can configure ONNX-related settings within your Spark session. This allows you to fine-tune the behavior of the ONNX engine for your specific needs.

Here are the available options for CPU:

- **intraOpNumThreads**: This setting, `spark.jsl.settings.onnx.intraOpNumThreads`, controls the number of threads used for intra-operation parallelism when executing ONNX models. You can set this value to optimize the performance of ONNX execution. To understand how this affects your ONNX tasks, refer to the ONNX documentation.

- **optimizationLevel**: Use `spark.jsl.settings.onnx.optimizationLevel` to specify the optimization level for ONNX execution. This setting influences how aggressively Spark NLP optimizes the execution of ONNX models. Explore the available options to determine which level suits your workload best in [this ONNX documentation]((https://onnxruntime.ai/docs/api/java/ai/onnxruntime/OrtSession.SessionOptions.OptLevel.html)).

- **executionMode**: With `spark.jsl.settings.onnx.executionMode`, you can choose the execution mode for ONNX models. Different modes may offer trade-offs between performance and resource utilization. Review the available options to select the mode that aligns with your requirements in [this ONNX documentation]((https://onnxruntime.ai/docs/api/java/ai/onnxruntime/OrtSession.SessionOptions.ExecutionMode.html))


Here are the available options for CUDA:

- **gpuDeviceId**: Use `spark.jsl.settings.onnx.gpuDeviceId` to define the GPU device to execute on

To find more information and detailed usage instructions for these ONNX configuration options, refer to the [ONNX API documentation](https://onnxruntime.ai/docs/api/).

In [ ]:
# Let's set our config based on our needs:
onnx_params = {
    "spark.jsl.settings.onnx.intraOpNumThreads": "5",
    "spark.jsl.settings.onnx.optimizationLevel": "BASIC_OPT",
    "spark.jsl.settings.onnx.executionMode": "SEQUENTIAL"
}

## Import AlbertForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `AlbertForQuestionAnswering` is only available since in `Spark NLP 5.1.1` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import ALBERT models trained/fine-tuned for question answering via `AlbertForQuestionAnswering`. These models are usually under `Question Answering` category and have `albert` in their labels
- Reference: [TFAlbertForQuestionAnswering](https://huggingface.co/transformers/model_doc/albert#transformers.TFAlbertForQuestionAnswering)
- Some [example models](https://huggingface.co/models?filter=albert&pipeline_tag=question-answering)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.29.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [1]:
!pip install -q --upgrade transformers[onnx]==4.29.1 optimum sentencepiece tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.1/396.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [twmkn9/albert-base-v2-squad2](https://huggingface.co/twmkn9/albert-base-v2-squad2) model from HuggingFace as an example and load it as a `ORTModelForQuestionAnswering`, representing an ONNX model.

In [2]:
from optimum.onnxruntime import ORTModelForQuestionAnswering
import tensorflow as tf

MODEL_NAME = 'twmkn9/albert-base-v2-squad2'
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)

# Save the ONNX model
ort_model.save_pretrained(EXPORT_PATH)

Framework not specified. Using pt to export to ONNX.


Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.0.1+cu118


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 46688
-rw-r--r-- 1 root root      844 Sep 13 19:31 config.json
-rw-r--r-- 1 root root 44758733 Sep 13 19:31 model.onnx
-rw-r--r-- 1 root root      173 Sep 13 19:31 special_tokens_map.json
-rw-r--r-- 1 root root   760289 Sep 13 19:31 spiece.model
-rw-r--r-- 1 root root      538 Sep 13 19:31 tokenizer_config.json
-rw-r--r-- 1 root root  2272267 Sep 13 19:31 tokenizer.json


- As you can see, we need to move `spiece.model` from the tokenizer to `assets` folder which Spark NLP will look for

In [4]:
!mkdir {EXPORT_PATH}/assets

In [5]:
!mv {EXPORT_PATH}/spiece.model {EXPORT_PATH}/assets

Voila! We have our `spiece.model` inside assets directory

In [6]:
!ls -lR {EXPORT_PATH}

onnx_models/twmkn9/albert-base-v2-squad2:
total 45948
drwxr-xr-x 2 root root     4096 Sep 13 19:31 assets
-rw-r--r-- 1 root root      844 Sep 13 19:31 config.json
-rw-r--r-- 1 root root 44758733 Sep 13 19:31 model.onnx
-rw-r--r-- 1 root root      173 Sep 13 19:31 special_tokens_map.json
-rw-r--r-- 1 root root      538 Sep 13 19:31 tokenizer_config.json
-rw-r--r-- 1 root root  2272267 Sep 13 19:31 tokenizer.json

onnx_models/twmkn9/albert-base-v2-squad2/assets:
total 744
-rw-r--r-- 1 root root 760289 Sep 13 19:31 spiece.model


## Import and Save AlbertForQuestionAnswering in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [9]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.1.1
setup Colab for PySpark 3.2.3 and Spark NLP 5.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.3/531.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 17.0 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function with the ONNX parameters that best fit our needs:

In [10]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start(params=onnx_params)

print("Apache Spark version: {}".format(spark.version))

- Let's use `loadSavedModel` functon in `AlbertForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `AlbertForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [11]:
from sparknlp.annotator import *
from sparknlp.base import *

spanClassifier = AlbertForQuestionAnswering.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [12]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [13]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your AlbertForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [14]:
! ls -l {MODEL_NAME}_spark_nlp_onnx

total 44472
-rw-r--r-- 1 root root 44765729 Sep 13 19:33 albert_classification_onnx
-rw-r--r-- 1 root root   760289 Sep 13 19:33 albert_spp
drwxr-xr-x 2 root root     4096 Sep 13 19:33 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [15]:
document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = AlbertForQuestionAnswering.load("./{}_spark_nlp_onnx".format(MODEL_NAME))\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([["What's my name?", "My name is Clara and I live in Berkeley."]]).toDF("question", "context")
result = pipeline.fit(example).transform(example)

result.select("answer.result").show(1, False)

+-------+
|result |
+-------+
|[clara]|
+-------+



That's it! You can now go wild and use hundreds of `AlbertForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
